In [ ]:
import os
source_dir = '.\\extracts'
target_dir = '.\\target'
all_files = []
for root, _, files in os.walk(source_dir):
        os.makedirs(root, exist_ok=True)
        for file in files:
            all_files.append(os.path.join(root, file))

print(os.path.splitdrive(os.path.abspath(target_dir)))

for file in all_files:
    print()
    print(file)
    print(os.path.dirname(file))
    print(os.path.basename(file))

    source_abs_path = os.path.abspath(file)
    src_rel_path = os.path.relpath(source_abs_path, source_dir)
    dest_rel_path = os.path.relpath(os.path.join(target_dir, src_rel_path))
    dest_abs_path = os.path.abspath(dest_rel_path)
    
    print(f"Source")
    print(f"abs: {source_abs_path}")
    print(f"rel: {src_rel_path}")

    print(f"Destination")
    print(f"rel: {dest_rel_path}")
    print(f"abs: {dest_abs_path}")


('c:', '\\rclone\\target')

.\src\sub1\another\test2.txt
.\src\sub1\another
test2.txt
Source
abs: c:\rclone\src\sub1\another\test2.txt
rel: sub1\another\test2.txt
Destination
rel: target\sub1\another\test2.txt
abs: c:\rclone\target\sub1\another\test2.txt

.\src\sub1\sub2\test.txt
.\src\sub1\sub2
test.txt
Source
abs: c:\rclone\src\sub1\sub2\test.txt
rel: sub1\sub2\test.txt
Destination
rel: target\sub1\sub2\test.txt
abs: c:\rclone\target\sub1\sub2\test.txt


In [ ]:
import shutil

source_dir = '.\\src'
operation = 'copy'  # or 'move'
target_dir = 'imported'
target_dir = os.path.join(os.path.expanduser('~\\Pictures'), target_dir)
is_same_drive = os.path.splitdrive(os.path.abspath(source_dir))[0].lower() == os.path.splitdrive(os.path.abspath(target_dir))[0].lower()

all_files = []
for root, _, files in os.walk(source_dir):
    for file in files:
        all_files.append(os.path.join(root, file))

target_dir = os.path.join(os.path.expanduser('~\\Pictures'), target_dir)
print(f"Target directory: {target_dir}")

# dest_file = os.path.join(target_dir, os.path.relpath(".\\src\\subfolder\\test.txt", start=source_dir))

for source_file in all_files:
    source_file = os.path.abspath(source_file)
    dest_file = os.path.join(target_dir, os.path.relpath(source_file, start=source_dir))
    print(f"Destination file: {dest_file}")
    if os.path.exists(dest_file):
        print(f"File already exists: {dest_file}. Skipping.")
        continue

    os.makedirs(os.path.dirname(dest_file), exist_ok=True)
    if operation == 'copy':
        shutil.copy2(source_file, dest_file)
        print(f"Copied: {source_file} to {dest_file}")
    else:  # move operation
        if is_same_drive:
            # On same drive, use rename (efficient move operation)
            os.rename(source_file, dest_file)
            print(f"Moved: {source_file} to {dest_file}")
        else:
            # Cross-drive move requires copy then delete
            shutil.copy2(source_file, dest_file)
            os.remove(source_file)
            print(f"Cross-drive move: {source_file} to {dest_file}")




Target directory: C:\Users\ddials\Pictures\imported
Destination file: C:\Users\ddials\Pictures\imported\sub1\another\test.txt
File already exists: C:\Users\ddials\Pictures\imported\sub1\another\test.txt. Skipping.
Destination file: C:\Users\ddials\Pictures\imported\sub1\another\test2.txt
File already exists: C:\Users\ddials\Pictures\imported\sub1\another\test2.txt. Skipping.
Destination file: C:\Users\ddials\Pictures\imported\sub1\sub2\test.txt
File already exists: C:\Users\ddials\Pictures\imported\sub1\sub2\test.txt. Skipping.
Destination file: C:\Users\ddials\Pictures\imported\sub1\sub2\test2.txt
File already exists: C:\Users\ddials\Pictures\imported\sub1\sub2\test2.txt. Skipping.


In [8]:
import os

source_dir = '.\\extracts\\Takeout\\Google Photos'
target_path = 'imported'
remote = 'onedrive'
rclone_path = 'rclone.exe'

try:
    # Build the rclone command with progress option
    cmd = [
        rclone_path, 
        "sync",  # Use 'sync' to mirror the source to the target
        # "--files-from", "upload_files_list.txt",
        "--progress",
        "--transfers", "4",  # Number of file transfers to run in parallel
        source_dir,
        f"{remote}:{target_path}"  # Destination
    ]
    
    print(f"Running command: {' '.join(cmd)}")
    import subprocess
    
    # Run the command and stream output in real-time
    process = subprocess.Popen(
        cmd,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        universal_newlines=True,
        bufsize=1
    )
    
    # Process and log output in real-time
    for line in process.stdout:
        line = line.strip()
        if line:
            if "ERROR" in line:
                print(f"ERROR: {line}")
            elif "transferred:" in line or "Transferred:" in line:
                # Highlight progress info
                print(f"INFO: {line}")
            else:
                print(f"DEBUG: {line}")

    process.wait()
    
    if process.returncode == 0:
        print(f"✓ Upload completed successfully")
    else:
        print(f"✗ Upload failed with return code {process.returncode}")

except Exception as e:
    print(f"ERROR: Error during upload: {str(e)}")
# finally:
#     # Clean up temp file
#     if os.path.exists(temp_list_file):
#         os.remove(temp_list_file)
#         print("DEBUG: Removed temporary file list")


Running command: rclone.exe sync --progress --transfers 4 .\extracts\Takeout\Google Photos onedrive:imported
INFO: Transferred:   	          0 B / 1.906 MiB, 0%, 0 B/s, ETA -
INFO: Transferred:            0 / 5, 0%
DEBUG: Elapsed time:         0.4s
DEBUG: Transferring:
DEBUG: *                                 IMG_8823.HEIC:  0% /1.906Mi, 0/s, -
DEBUG: *                         sub1/another/test.txt:  0% /0, 0/s, -
DEBUG: *                            sub1/sub2/test.txt:  0% /0, 0/s, -
INFO: *                           sub1/sub2/test2.txt:  0% /0, 0/s, -Transferred:   	          0 B / 1.906 MiB, 0%, 0 B/s, ETA -
INFO: Transferred:            0 / 5, 0%
DEBUG: Elapsed time:         0.9s
DEBUG: Transferring:
DEBUG: *                                 IMG_8823.HEIC:  0% /1.906Mi, 0/s, -
DEBUG: *                         sub1/another/test.txt:  0% /0, 0/s, -
DEBUG: *                            sub1/sub2/test.txt:  0% /0, 0/s, -
INFO: *                           sub1/sub2/test2.txt:  0% /0, 0/s, 

In [ ]:
import os